# Make graphs for the publication in order to visualize the differences between populations 
## Author: Efthymios Tzinis 

In [ ]:
# read the data 
datapath = '../data/distance_pairs_remastered.xlsx'
resultpath = '../results/plots/'

import pandas as pd
from pprint import pprint
xls = pd.ExcelFile(datapath)
df = xls.parse(xls.sheet_names[0])
df.dropna(inplace=True)
print df

In [ ]:
# make a dictionary for all populations baased on stats values 

pval = df.loc[0:7]
pval.set_index(['p-value'],inplace=True)
pval = pval.to_dict()

oddsratio = df.loc[9:]
oddsratio.set_index(['p-value'],inplace=True)
oddsratio = oddsratio.to_dict()

pprint (pval)
# pprint (oddsratio)

In [ ]:
# plot a radian evenly distributed graph
from plotly.graph_objs import *
import plotly
plotly.offline.init_notebook_mode()
import math

def plot_graph_in_radius_form(dic,title):
    
    angle = (2 *math.pi) / len(dic)
    nodes_size = 50
    data = []
    circles = []
    radius_offset = 0.25
    
    back_circles = 5
    how_many = 7
    for i in range(how_many):
        circle ={
            'opacity': .5,
            'type': 'circle',
            'xref': 'x',
            'yref': 'y',
            'x0': float(-(radius_offset +  i*1.0  / back_circles)),
            'y0': float(-(radius_offset +  i*1.0 / back_circles)),
            'x1': float(radius_offset +  i*1.0  / back_circles),
            'y1': float(radius_offset + i*1.0 / back_circles),
            'line': {
                'color': 'rgba(190, 190, 190, 1)',
            },
        }
        circles.append(circle)
        
    trace0 = Scatter(
        x = [0],
        y = [0],
        name = 'SEC',
        text = ['SEC'],
        mode = 'markers+text',
        textposition='center',
        marker = dict(
            size = [nodes_size]
        )
    )
    
    data.append(trace0)
    
    position_ang = 0
    for p,r in dic.items():
        new_r = r + radius_offset
        trace = plotly.graph_objs.Scatter(
            x = [new_r * math.cos(position_ang)],
            y = [new_r * math.sin(position_ang)],
            name = p,
            text = [p],
            mode = 'markers+text',
            textposition='center',
            marker = dict(
                size = [nodes_size]
            )
        )
        
        data.append(trace)
        position_ang += angle

    layout = dict(title = title,
                  showlegend = False,
                    shapes = circles,
                  autosize=False,
                    width=600,
                    height=600,
                  yaxis = dict(autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False),
                  xaxis = dict(autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False)
                 )

    fig = plotly.graph_objs.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig, filename= title)



In [ ]:
def compute_radius_pval(dic):
    new_dic = {}
    for k in dic:
        number = str(dic[k])
        dist = 1 - float(number.strip("<"))
        new_dic[k] = dist
    return new_dic

# all comparisons are made between: SEC=Southeastern European Caucasian population and other populations  
for polym in pval:
#     for this polymorphism we create the distance graph based on p-value
#     big values for p are related to high
    distances_dic = compute_radius_pval(pval[polym])
    plot_graph_in_radius_form(distances_dic,polym)

In [ ]:
# compute the overall distance from alla the polymorphisms
n=0.0
overall_dic = None
for polym in pval:
#     faggreagate distances
    distances_dic = compute_radius_pval(pval[polym])
    if overall_dic is not None:
        for k in overall_dic:
            overall_dic[k] += distances_dic[k]
    else:
        overall_dic = distances_dic
    n+=1 
for k in overall_dic:
    overall_dic[k] /= n
# print overall_dic
plot_graph_in_radius_form(overall_dic,'Aggregated similarity from all polymorphisms')